# Feed Forward Neural Network 

In [3]:
it_clust_dataset_train = '/dataset/Baron_pancreatic_islet.h5ad'
it_clust_dataset_test = '/dataset/smartseq2.h5ad'

In [4]:
!pip3 install scanpy pandas numpy matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 5.7 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=45431b0da7986c7b77eaccdd6df40a85ba851c46f5b61c11625bb7ef7e865004
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=1b2851291f470b551e

In [5]:
import pickle

In [6]:
import scanpy as sc
import os
from numpy.random import seed
# from tensorflow import set_random_seed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [7]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
import numpy as np
# from fancyimpute import IterativeImputer

In [8]:
adata_train=sc.read(it_clust_dataset_train)
adata_test=sc.read(it_clust_dataset_test)

In [9]:
len(adata_train.obs["celltype"])

8569

In [10]:
#/Utils/preprocess.py
def preprocess(adata_test, min_genes = 200, min_cells = 20, target_sum = 1e6, n_top_genes = 3000, max_value = 10, get_hvgs=False, scale_and_hvgs = False):
        """
        INPUT:
        file_path: path to .h5ad containing scRNA-seq
        """
        ## convert to h5ad
        # adata_test = sc.AnnData(genes, labels)

        ## make var names unique
        adata_test.obs_names_make_unique()
        adata_test.var_names_make_unique()

    ## filter cells with count less than 200
        sc.pp.filter_cells(adata_test, min_genes=min_genes)

        ## filter genes with count less than 20
        sc.pp.filter_genes(adata_test, min_cells=min_cells)

        ## normalise data
        sc.pp.normalize_total(adata_test, target_sum=target_sum)

        ## LogNormalise
        if not(scale_and_hvgs):
                return {'data':adata_test}

        if get_hvgs:
                ## Get HVGS
                sc.pp.log1p(adata_test)
                sc.pp.highly_variable_genes(adata_test, n_top_genes = n_top_genes)
                adata_test = adata_test[:, adata_test.var.highly_variable]

                ## scale data
                sc.pp.scale(adata_test, max_value=max_value)
                return {'data' : adata_test, 'hvg': adata_test.var.highly_variable}

        ## scale data
        sc.pp.scale(adata_test, max_value=max_value)
        return {'data':adata_test}

In [11]:
print("Starting preprocessing...")
train_dic = preprocess(adata_train, min_cells=20,min_genes=50, get_hvgs = True, scale_and_hvgs = True)
test_dic = preprocess(adata_test, min_cells=0, min_genes=0)

Starting preprocessing...


/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [12]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
print(len(intersection(adata_train.var.index, adata_test.var.index)))
list(adata_train.var.index)[0]

14260


'A1BG'

In [13]:
col= [i for i in train_dic['hvg'].index]
print(col)

['A2M', 'A4GALT', 'AADAC', 'AATK', 'ABCA4', 'ABCA8', 'ABCB1', 'ABCC3', 'ABCC8', 'ABCC9', 'ABHD15', 'ABHD2', 'ABHD4', 'ABI3', 'ABI3BP', 'ABL1', 'ABL2', 'ABTB2', 'ACAN', 'ACE', 'ACKR3', 'ACOXL', 'ACP5', 'ACRC', 'ACSL1', 'ACSL4', 'ACSL5', 'ACSS3', 'ACTA1', 'ACTA2', 'ACTB', 'ACTC1', 'ACVRL1', 'ADA', 'ADAM12', 'ADAM19', 'ADAM8', 'ADAMDEC1', 'ADAMTS1', 'ADAMTS12', 'ADAMTS14', 'ADAMTS2', 'ADAMTS4', 'ADAMTS5', 'ADAMTS7', 'ADAMTS9', 'ADAMTSL2', 'ADAP2', 'ADCY1', 'ADCY2', 'ADCY3', 'ADCY4', 'ADCY5', 'ADCY7', 'ADCY9', 'ADCYAP1', 'ADD2', 'ADD3', 'ADGRA2', 'ADGRD1', 'ADGRE2', 'ADGRF1', 'ADGRF5', 'ADGRG1', 'ADGRL2', 'ADGRL4', 'ADH1C', 'ADH6', 'ADIRF', 'ADM', 'ADM5', 'ADNP2', 'ADORA1', 'ADORA3', 'ADRA2C', 'ADRB2', 'AEBP1', 'AEN', 'AFAP1', 'AFAP1L1', 'AFAP1L2', 'AFF2', 'AGAP1', 'AGER', 'AGR2', 'AGR3', 'AGRN', 'AGT', 'AGTR1', 'AGXT', 'AHNAK', 'AHR', 'AHRR', 'AIF1', 'AIM1', 'AJAP1', 'AK4', 'AK8', 'AKAP12', 'AKAP7', 'AKNA', 'AKR1B1', 'AKR1B10', 'AKR1C1', 'AKR1C2', 'AKR1C3', 'ALB', 'ALCAM', 'ALDH1A1', 'ALD

In [14]:
train_adata_pp =  train_dic['data']
print(train_dic['hvg'])
test_adata_pp =  test_dic['data'][:, intersection(col, test_dic['data'].var.index)]
train_adata_pp = train_dic['data'][:, intersection(col, train_dic['data'].var.index)]

index
A2M       True
A4GALT    True
AADAC     True
AATK      True
ABCA4     True
          ... 
ZSCAN9    True
ZSWIM3    True
ZSWIM4    True
ZYX       True
pk        True
Name: highly_variable, Length: 3000, dtype: bool


In [15]:
train_df = train_adata_pp.to_df()
test_df = test_adata_pp.to_df()

In [16]:
## taking common genes
print("Taking common genes...")
final_columns = list(set(train_df.columns).intersection(set(test_df.columns)))
print('Common columns', len(final_columns))
final_columns = [i for i in final_columns if i != 'celltype'] 
train_df = train_df[final_columns]
test_df = test_df[final_columns]

Taking common genes...
Common columns 2975


In [17]:
y_train = train_adata_pp.obs.celltype.to_list()
y_test = test_adata_pp.obs.celltype.to_list()

In [18]:
print(len(y_train))
print(len(y_test))

8569
2394


In [19]:
freq = {}
for item in y_test:
    if (item in freq):
        freq[item] += 1
    else:
        freq[item] = 1

In [20]:
print(freq)

{'gamma': 213, 'acinar': 188, 'alpha': 1008, 'delta': 127, 'beta': 308, 'ductal': 444, 'endothelial': 21, 'activated_stellate': 55, 'schwann': 2, 'mast': 7, 'macrophage': 7, 'epsilon': 8, 'quiescent_stellate': 6}


In [21]:
X_train = train_df.to_numpy()
X_test = test_df.to_numpy()

In [22]:
import pickle

In [23]:
%ls

drive/  sample_data/


In [24]:
labels = set(y_train)
mapping = {}
cnt = 0
for lab in set(y_train):
  if lab in mapping:
    continue
  mapping[lab] = cnt
  cnt += 1


In [25]:
print(mapping)

{'epsilon': 0, 't_cell': 1, 'activated_stellate': 2, 'alpha': 3, 'quiescent_stellate': 4, 'macrophage': 5, 'gamma': 6, 'ductal': 7, 'beta': 8, 'endothelial': 9, 'delta': 10, 'acinar': 11, 'schwann': 12, 'mast': 13}


In [26]:
y_test_lab = []
for i in y_test:
  if i in mapping:
    y_test_lab.append(mapping[i])
  else:
    y_test_lab.append(0)

In [27]:
y_test = np.array(y_test_lab)

In [28]:
y_train_lab = []
for i in y_train:
  if i in mapping:
    y_train_lab.append(mapping[i])
  else:
    y_train_lab.append(0)

In [29]:
y_train = np.array(y_train_lab)

In [30]:
y_train

array([11, 11, 11, ...,  8,  8,  7])

In [31]:
X_train.shape

(8569, 2975)

In [32]:
X_test.shape

(2394, 2975)

In [33]:
len(y_test)

2394

In [34]:
len(y_train)

8569

In [35]:
from skllm.config import SKLLMConfig
SKLLMConfig.set_openai_key("")
SKLLMConfig.set_openai_org("")

In [36]:
from skllm import ZeroShotGPTClassifier

In [37]:
clf = ZeroShotGPTClassifier(openai_model = "gpt-3.5-turbo")


In [38]:
clf.fit(X_train, y_train)

ZeroShotGPTClassifier()

In [39]:
labels = clf.predict(X_test)

100%|██████████| 2394/2394 [07:09<00:00,  5.57it/s]


In [40]:
labels

[11,
 2,
 3,
 3,
 10,
 11,
 8,
 8,
 9,
 3,
 9,
 3,
 2,
 3,
 8,
 7,
 8,
 9,
 8,
 8,
 7,
 6,
 9,
 10,
 9,
 7,
 3,
 8,
 8,
 3,
 7,
 3,
 11,
 8,
 8,
 7,
 8,
 4,
 8,
 11,
 3,
 3,
 10,
 8,
 8,
 7,
 3,
 8,
 10,
 3,
 3,
 11,
 11,
 11,
 2,
 7,
 8,
 8,
 3,
 3,
 3,
 6,
 9,
 8,
 8,
 3,
 8,
 8,
 1,
 11,
 3,
 3,
 3,
 2,
 8,
 7,
 10,
 3,
 11,
 3,
 7,
 3,
 8,
 3,
 3,
 7,
 8,
 3,
 11,
 8,
 8,
 3,
 3,
 8,
 11,
 2,
 8,
 3,
 3,
 3,
 3,
 7,
 8,
 2,
 11,
 8,
 11,
 9,
 8,
 8,
 10,
 11,
 3,
 3,
 11,
 11,
 3,
 3,
 3,
 8,
 3,
 11,
 11,
 8,
 8,
 3,
 3,
 11,
 3,
 11,
 3,
 8,
 8,
 8,
 7,
 8,
 10,
 8,
 3,
 8,
 8,
 8,
 8,
 2,
 2,
 3,
 3,
 10,
 8,
 3,
 11,
 3,
 6,
 7,
 8,
 3,
 7,
 8,
 11,
 3,
 11,
 11,
 8,
 7,
 3,
 7,
 11,
 8,
 7,
 8,
 8,
 3,
 6,
 9,
 7,
 8,
 3,
 11,
 8,
 8,
 7,
 8,
 8,
 7,
 8,
 5,
 3,
 11,
 7,
 8,
 3,
 7,
 8,
 7,
 9,
 3,
 7,
 6,
 3,
 3,
 8,
 7,
 8,
 7,
 10,
 3,
 8,
 8,
 10,
 3,
 11,
 7,
 7,
 3,
 0,
 7,
 8,
 11,
 10,
 10,
 3,
 3,
 11,
 8,
 8,
 2,
 7,
 3,
 8,
 3,
 7,
 8,
 7,
 10,
 7,
 3,
 8,
 8,
 8,
 

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, labels, target_names=list(mapping.keys())))

                    precision    recall  f1-score   support

           epsilon       0.00      0.00      0.00         8
            t_cell       0.00      0.00      0.00         0
activated_stellate       0.03      0.04      0.03        55
             alpha       0.42      0.28      0.34      1008
quiescent_stellate       0.00      0.00      0.00         6
        macrophage       0.00      0.00      0.00         7
             gamma       0.06      0.02      0.03       213
            ductal       0.19      0.14      0.16       444
              beta       0.12      0.27      0.16       308
       endothelial       0.03      0.10      0.04        21
             delta       0.05      0.06      0.06       127
            acinar       0.08      0.11      0.09       188
           schwann       0.00      0.00      0.00         2
              mast       0.00      0.00      0.00         7

          accuracy                           0.19      2394
         macro avg       0.07      0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
from skllm.preprocessing import GPTVectorizer

In [44]:
!pip install scgpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 638.6/638.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 